In [1]:
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle
from utils import clean_pattern, define_network

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
with open('intents.json') as file:
    data = json.load(file)
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'info', 'patterns': ['how you born', 'what is used to create you', 'what technology used for you?'], 'responses': ['Using tensorflow and tflearn you can create a simple intent based chatbot!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?', 'what are you?', 'what do you do?'], 'responses': ["I'm a chatbot created by adiprogrammer", 'A chatbot program, I talk to the people :)', "I'm a lower level jarvis/intent based chatbot"], 'context_set': ''}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "Th

In [3]:
# Some cleaning of data in intents.json
stemmed_words = []
tags = []
ignore_words = ['!', '?', '.']
corpus = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        stemmed_pattern = clean_pattern(pattern, ignore_words)
        stemmed_words.extend(stemmed_pattern)
        corpus.append((stemmed_pattern, intent['tag']))
    if intent['tag'] not in tags:
        tags.append(intent['tag'])

# remove duplicates and sort
stemmed_words = sorted(list(set(stemmed_words)))
tags = sorted(list(set(tags)))

print(stemmed_words)
print(tags)
print(corpus)

["'m", "'s", ',', 'a', 'am', 'anyon', 'ar', 'awesom', 'bor', 'born', 'cal', 'cre', 'cya', 'day', 'do', 'for', 'good', 'goodby', 'hard', 'hav', 'hello', 'help', 'hi', 'how', 'i', 'is', 'lat', 'leav', 'lif', 'liv', 'me', "n't", 'na', 'nam', 'sad', 'see', 'should', 'technolog', 'thank', 'that', 'ther', 'to', 'unhappy', 'up', 'us', 'wan', 'what', 'yo', 'you']
['funny', 'goodbye', 'greeting', 'info', 'name', 'thanks']
[(['hi'], 'greeting'), (['how', 'ar', 'you'], 'greeting'), (['is', 'anyon', 'ther'], 'greeting'), (['hello'], 'greeting'), (['good', 'day'], 'greeting'), (['what', 'up'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you', 'lat'], 'goodbye'), (['goodby'], 'goodbye'), (['i', 'am', 'leav'], 'goodbye'), (['hav', 'a', 'good', 'day'], 'goodbye'), (['how', 'you', 'born'], 'info'), (['what', 'is', 'us', 'to', 'cre', 'you'], 'info'), (['what', 'technolog', 'us', 'for', 'you'], 'info'), (['what', 'is', 'yo', 'nam'], 'name'), (['what', 'should', 'i', 'cal', 'you'], 'name'), (['what', 'yo

In [4]:
# Creating numeric features and labels out of cleaned data
X = []
y = []
for item in corpus:
    bag = [] #array of 1 and 0. 1 if stemmed word is present stemmed pattern
    stemmed_pattern = item[0]
    for w in stemmed_words:
        if w in stemmed_pattern:
            bag.append(1)
        else:
            bag.append(0)

    tags_row = [] #array of 1 and 0. 1 for current tag and for everything else 0.
    current_tag = item[1]
    for tag in tags:
        if tag == current_tag:
            tags_row.append(1)
        else:
            tags_row.append(0)

    #for each item in corpus, X will be array indicating stemmed words and y array indicating tags
    X.append(bag)
    y.append(tags_row) 

X = np.array(X)
y = np.array(y)
print(X)
print(y)

# saving variables in pickle to be used by main.py
with open('saved_variables.pickle', 'wb') as file:
    pickle.dump((stemmed_words, tags, ignore_words, X, y), file)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
[[0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]


In [5]:
# our model stuff from main.py
model = define_network(X, y)
model.fit(X, y, n_epoch=600, batch_size=8, show_metric=True, shuffle= True) #n_epoch:no. of times model will see same data
model.save("chatbot_model.tflearn")

Training Step: 2399  | total loss: 0.14940 | time: 0.024s
| Adam | epoch: 600 | loss: 0.14940 - acc: 0.9876 -- iter: 24/29
Training Step: 2400  | total loss: 0.14020 | time: 0.032s
| Adam | epoch: 600 | loss: 0.14020 - acc: 0.9888 -- iter: 29/29
--
INFO:tensorflow:F:\ChatBot\chatbot_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
